In [69]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

Calculation started (calculation_id=06c4edf0-62dd-d93f-4f93-af52e831cf52) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [70]:
df_bancos_raw = spark.read.csv("s3://ingestao-dados/raw/Bancos/", sep='\t', header=True)

Calculation started (calculation_id=aec4edf0-682f-9afc-75cf-b93bc48cb7ce) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [71]:
df_empregados_raw = spark.read.csv("s3://ingestao-dados/raw/Empregados/", sep='|', header=True).dropDuplicates(["Nome"])

Calculation started (calculation_id=ccc4edf0-767c-f4d7-272e-9cadedfcd4da) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [72]:
df_reclamacoes_raw = spark.read.csv("s3://ingestao-dados/raw/Reclamacoes_utf8/", sep=';', header=True, encoding='UTF-8')

Calculation started (calculation_id=d6c4edf0-7bf0-b90a-1fd0-023ac04b1fb2) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [73]:
df_bancos = df_bancos_raw.select(
    F.col("Segmento").alias("segmento"),
    F.lpad(F.col("CNPJ"), 8, '0').alias("cnpj"),
    F.regexp_replace(F.col("Nome"), ' - PRUDENCIAL', '').alias("nome")
)

Calculation started (calculation_id=16c4edf0-8dcb-5b45-1aa9-936e17d84206) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [74]:
df_empregados = df_empregados_raw.select(
    F.col("employer_name").cast(StringType()).alias("employer_name"),
    F.col("reviews_count").cast(LongType()).alias("reviews_count"),
    F.col("culture_count").cast(LongType()).alias("culture_count"),
    F.col("salaries_count").cast(LongType()).alias("salaries_count"),
    F.col("benefits_count").cast(LongType()).alias("benefits_count"),
    F.col("employer-website").cast(StringType()).alias("employer_website"),
    F.col("employer-headquarters").cast(StringType()).alias("employer_headquarters"),
    F.col("employer-founded").cast(IntegerType()).alias("employer_founded"),
    F.col("employer-industry").cast(StringType()).alias("employer_industry"),
    F.col("employer-revenue").cast(StringType()).alias("employer_revenue"),
    F.col("url").cast(StringType()).alias("url"),
    F.col("Geral").cast(FloatType()).alias("geral"),
    F.col("Cultura e valores").cast(FloatType()).alias("cultura_valores"),
    F.col("Diversidade e inclus�o").cast(FloatType()).alias("diversidade_inclusao"),
    F.col("Qualidade de vida").cast(FloatType()).alias("qualidade_vida"),
    F.col("Alta lideran�a").cast(FloatType()).alias("alta_lideranca"),
    F.col("Remunera��o e benef�cios").cast(FloatType()).alias("remuneracao_beneficios"),
    F.col("Oportunidades de carreira").cast(FloatType()).alias("oportunidades_carreira"),
    F.col("Recomendam para outras pessoas(%)").cast(FloatType()).alias("recomendam_outras_pessoas"),
    F.col("Perspectiva positiva da empresa(%)").cast(FloatType()).alias("perspectiva_positiva_empresa"),
    F.col("Segmento").cast(StringType()).alias("segmento"),
    F.col("Nome").cast(StringType()).alias("nome"),
    F.col("match_percent").cast(FloatType()).alias("match_percent")
)

Calculation started (calculation_id=52c4edf0-9341-3d14-f029-3b3d953de4da) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [75]:
df_bancos_empregados = df_bancos.join(df_empregados, ['nome', 'segmento'], 'left')

Calculation started (calculation_id=9ec4edf0-98a3-28a7-6194-0d3ecb5bbea9) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [76]:
df_reclamacoes = df_reclamacoes_raw.withColumn("cnpj", F.when(
    F.col("CNPJ IF") == ' ', F.lit(None)
).otherwise(F.col("CNPJ IF")))\
.select(
    F.col("Ano").cast(IntegerType()).alias("ano"),
    F.col("Trimestre").cast(StringType()).alias("trimestre"),
    F.col("Categoria").cast(StringType()).alias("categoria"),
    F.col("Tipo").cast(StringType()).alias("tipo"),
    F.lpad(F.col("cnpj"), 8, '0').alias("cnpj"),
    F.col("Institui��o financeira").cast(StringType()).alias("instituicao_financeira"),
    F.col("�ndice").cast(FloatType()).alias("indice"),
    F.col("Quantidade de reclama��es reguladas procedentes").cast(LongType()).alias("qtd_reclamacoes_reguladas_procedentes"),
    F.col("Quantidade de reclama��es reguladas - outras").cast(LongType()).alias("qtd_reclamacoes_reguladas_outras"),
    F.col("Quantidade de reclama��es n�o reguladas").cast(LongType()).alias("qtd_reclamacoes_nao_reguladas"),
    F.col("Quantidade total de reclama��es").cast(LongType()).alias("qtd_total_reclamacoes"),
    F.col("Quantidade total de clientes - CCS e SCR").cast(LongType()).alias("qtd_total_clientes_ccs_scr"),
    F.col("Quantidade de clientes - CCS").cast(LongType()).alias("qtd_clientes_ccs"),
    F.col("Quantidade de clientes - SCR").cast(LongType()).alias("qtd_clientes_scr"),
    F.regexp_replace(F.col("Institui��o financeira"), " \\(conglomerado\\)", "").alias("nome")
)

Calculation started (calculation_id=76c4edf0-9a06-ad3f-d1d9-71847a7ec9a5) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [77]:
df_nome = df_bancos_empregados.join(df_reclamacoes.where(F.col("cnpj").isNull()).drop("cnpj"), ['nome'], 'left')

df_nome = df_nome.select(
    F.col("nome"),
    F.col("cnpj"),
    F.col("segmento").alias("classificacao_banco"),
    F.col("geral").alias("avaliacao_geral"),
    F.col("indice").alias("indice_reclamacoes"),
    F.col("qtd_total_reclamacoes"),
    F.col("qtd_total_clientes_ccs_scr"),
    F.col("remuneracao_beneficios").alias("indice_satisfacao_salarios_funcionarios"),
    F.col("perspectiva_positiva_empresa"),
    F.col("ano"),
    F.col("trimestre")
)

Calculation started (calculation_id=9ac4edf0-9b86-2683-0d06-87231e17af98) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [78]:
df_nome.count()

Calculation started (calculation_id=e2c4edf0-9ced-2513-ef86-a934abf14b32) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
1691



In [79]:
df_cnpj = df_bancos_empregados.join(df_reclamacoes.where(F.col("cnpj").isNotNull()).drop("nome"), ['cnpj'], 'left')
df_cnpj = df_cnpj.select(
    F.col("nome"),
    F.col("cnpj"),
    F.col("segmento").alias("classificacao_banco"),
    F.col("geral").alias("avaliacao_geral"),
    F.col("indice").alias("indice_reclamacoes"),
    F.col("qtd_total_reclamacoes"),
    F.col("qtd_total_clientes_ccs_scr"),
    F.col("remuneracao_beneficios").alias("indice_satisfacao_salarios_funcionarios"),
    F.col("perspectiva_positiva_empresa"),
    F.col("ano"),
    F.col("trimestre")
)

Calculation started (calculation_id=10c4edf0-b2f8-c973-e864-8b0a8a00c079) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [80]:
df_cnpj.count()

Calculation started (calculation_id=7ec4edf0-b464-8b5c-b330-81f4725c5896) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
1728



In [81]:
df = df_nome.unionByName(df_cnpj).dropDuplicates()

Calculation started (calculation_id=38c4edf0-ca43-9108-357f-b6b48e47b8f1) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [82]:
df.count()

Calculation started (calculation_id=a2c4edf0-cbc0-aa87-09bd-7d0449e7f819) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
2064



In [83]:
df.repartition(1).write.mode("overwrite").parquet("s3://ingestao-dados/transformed_pyspark/")

Calculation started (calculation_id=d2c4edf0-d54f-4e09-a5f2-1bfecd2b3f37) in (session=02c4ede6-db86-afe5-b5cf-dc0e58b733bc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
